## Hands on session 2.4 - highly composite derivations

This session returns to the calculation tree derivations first shown in session 1.3, building on these tools and demonstrating a real use case for them.

The file `anisotropy.py` is supplied which contains a collection of functions used in more complex derivations.

Demonstrated concepts:

- Building functions out of node calculations
- Applying range filter cutoffs in derivations
- Additive derivations
- Implicit temperature derivations


In [28]:
import numpy as np

import holoviews as hv
import matplotlib.pyplot as plt
import panel as pn

import anisotropy as aniso
import RMK_support.simple_containers as sc
import RMK_support.common_models as cm
import RMK_support.IO_support as io
from RMK_support import RKWrapper, Grid, treeDerivation, Node
import RMK_support.dashboard_support as ds

### The system
A true model of an anisotropic plasma would require a full set of fluid equations. For the sake of brevity this is not done here. Nonetheless, one integral part of an anisotropic plasma model is the isotropisation of temperatures. It is this process that is demonstrated here in this notebook.

The equations shown here can be found in [Chodura et al.](link)

We will look at the simplified one species case (for electrons) first. In this case, the anisotropy is quantified by the variable 

$$
    X = \frac{T_\perp}{T_\parallel} - 1
$$

With these variables defined, the equations that govern the system are:
$$
    \frac{\partial W_\perp}{\partial t} = 2 n \nu \left[T_\perp (K_{002} - K_{200}) \right]
$$
$$
    \frac{\partial W_\parallel}{\partial t} = 4 n \nu \left[ T_\parallel (K_{200} - K_{002}) \right]
$$
where $\nu$ is in our case an arbitrary collisionality.
$K_{LMN}$ are results of taking moments of a bi-Maxwellian:
$$
    K_{200} = \frac{1}{X} \left[ -1 + (1 + X)\varphi (X) \right]
$$
$$
    K_{002} = \frac{2}{X} \left[ 1 - \varphi (X) \right]
$$
where $\varphi$ for $X>0$:
$$
    \varphi = \frac{\arctan{(\sqrt{X})}}{\sqrt{X}}
$$
and for $X<0$:
$$
    \varphi = \frac{\log{\frac{1+\sqrt{-X}}{1-\sqrt{-X}}}}{2\sqrt{-X}}
$$

### Basic setup

In a full model evolving all fluid variables we would include many spatial derivative terms. However, since we're only interested in the 0D effect of temperature isotropisation we can reduce the number of cells simulated as much as possible. Because we want to write the setup in a general way, using staggered grids, we need at least 2 cells.

In [29]:
#Wrapper initialization
rk = RKWrapper()

#I/O setuo
rk.jsonFilepath = "./config.json" 
hdf5Filepath = "./RMKOutput/day_2_4/"
rk.setHDF5Path(hdf5Filepath) 

#Grid initialization
xGridWidths = np.ones(2)
rk.grid = Grid(xGridWidths, interpretXGridAsWidths=True)

### Normalization
Alongside the standard ReMKiT1D normalisation parameters that are employed, it is worthwhile specifying the electron mass.

In [30]:
elMass = 9.10938e-31 # electron mass
amu = 1.6605390666e-27  # atomic mass unit

rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

### Handling particle species data
As covered in workshop session 2.3, some built in models require knowledge of the species they are being constructed for. This is the case for the `implicitTemperatures()` model that is used in this notebook. As such, let us declare an electron species that will contain all of the associated implicit variables.

In [31]:
rk.addSpecies("e", speciesID=0, atomicA=elMass/amu, charge=-1.0, associatedVars=["ne", "Ge", "WePar", "WePerp"])

electronSpecies = rk.getSpecies("e")

### Base variables
With the species now declared, it is time to introduce the base variables of the system. These are the implicit variables upon which the models and their respective derived terms are built upon. Before adding the variables to the wrapper, we shall create some initial conditions that will be assigned to the different variables. 

**NOTE** In an isotropic system the internal energy of an ideal gas can be written as $W = \frac32 n T$ (in normalised units). But due to the anisotropic nature of this system we must separate the degrees of freedom, hence $W_\perp = n T_\perp$ and $W_\parallel = \frac12 n T_\parallel$.

In [32]:
# Set conserved variables in container
TePar = 2*np.ones(2)
TePerp = 5*np.ones(2)

ne = np.ones(2)

WePar = ne*TePar/2
WePerp = ne*TePerp

WIth these initial conditions declared, we can create variables for density, flux and internal energy:

In [33]:
rk.addVarAndDual("ne", ne, isCommunicated=True)
rk.addVarAndDual("Ge", primaryOnDualGrid=True, isCommunicated=True)  # Ge_dual is evolved, and Ge is derived
rk.addVarAndDual("WePar", WePar, isCommunicated=True)
rk.addVarAndDual("WePerp", WePerp, isCommunicated=True)

We would like to have the temperatures implicit, but they should be related to the corresponding energy densities. Hence we label them as stationary variables, which can then be 'evolved' through the `implicitTemperatures()` model.

In [ ]:
rk.addVarAndDual("TePar", TePar, isStationary=True, isCommunicated=True)
rk.addVarAndDual("TePerp", TePerp, isStationary=True, isCommunicated=True)

### Anisotropic variables
The anisotropic variables that are derived using the implicit variables declared above are: $\nu$, $K_{200}$ & $K_{002}$. We can now add these to the ReMKiT1D wrapper:

In [34]:
rk.addVar("collFreq", np.ones(2),isDerived=True) # Collision frequency set arbitrarily

rk.addCustomDerivation("K200", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),LMN="200")))
rk.addCustomDerivation("K002", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),LMN="002")))

There is one issue with these terms, namely as the system approaches isotropy, the value of $X$ will aproach zero. This is problematic as the numerical solver will break due to a divide by zero error. Hence, the terms $K_{200}$ and $K_{002}$ need to have a form that is used only when X approaches zero. This situation is a prime candidate to make use of ReMKiT1D's `rangeFilterDerivation()` function. 

To begin, let's create the variable $X$ as well as derivations for $K_{200}$ $K_{002}$ when $X$ is small:

In [35]:
rk.addVarAndDual("X",
                 isDerived=True,
                 derivationRule=sc.derivationRule("XDeriv", ("TePar","TePerp")),
                 derivOptions=treeDerivation(Node("TePerp")/Node("TePar") - 1)) # YOUR CODE HERE?

rk.addCustomDerivation("K200Small", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),LMN="200",smallX=True)))

rk.addCustomDerivation("K002Small", treeDerivation(aniso.K_LMN(Node("TePerp")/Node("TePar"),LMN="002",smallX=True)))

We can use the `rangeFilterDerivation()` function found in `simple_containers` to write what the values of $K_{200}$ and $K_{002}$ are in the different regimes. Since $-\infty < X < \infty$ we must assign three different ranges in which $K_{200}$ and $K_{002}$ must be calculated. These being:
$$
    \begin{cases}
        -1e16 < X < -0.01 \\
        |X| << 0.01 \\
        0.01 > X < 1e16 \\
    \end{cases}
$$
**Note** that the range is not actually $-\infty < X < \infty$, since the solver deals with floating point numbers we must give a large float as the range.

The `rangeFilterDerivation()` is passed a list of variables that are required to create the derivation. Firstly the derivation that must be applied in the given range must be provided. The control variable that is used to specify the range must be provided, this is done by passing the index of the control variable from the list that is passed to the `rangeFilterDerivation()`. Next the range of values that the control variable may have is specified. The variables that are required by the derivation are specified, this is done by providing the index of the variables used in the derivation.

This is also explained in the docstring of the `rangeFilterDerivation()`:

In [36]:
help(sc.rangeFilterDerivation)

Help on function rangeFilterDerivation in module RMK_support.simple_containers:

rangeFilterDerivation(derivName: str, controlIndices: List[int], controlRanges: List[List[float]], derivIndices: Optional[List[int]] = None) -> dict
    Return composite derivation object which wraps another derivation with range-based filtering, zeroing out all values where
    passed variables corresponding to controlIndices are outside ranges specified by controlRanges. If derivIndices aren't present all
    passed variables are passed to the derivation in that order.
    
    Args:
        derivName (str): Name of the wrapped derivation
        controlIndices (List[int]): Indices of passed variables corresponding to control variables
        controlRanges (list[List[float]]): Ranges (all length 2) corresponding to each control variable
        derivIndices (Union[None,List[int]], optional): Optional subset of passed variables passed to the wrapped derivation. Defaults to None, passing all variables.
  

We add a derivation for each interval for both $K_{200}$ and $K_{002}$. Making sure that for small values, the correct derivation is used.

In [37]:
rk.addCustomDerivation("filterSmallValsK200", sc.rangeFilterDerivation("K200Small",controlIndices=[1],controlRanges=[[-0.01,0.01]],derivIndices=[2,3]))
rk.addCustomDerivation("filterPlusLargeK200", sc.rangeFilterDerivation("K200",controlIndices=[1],controlRanges=[[0.01,1e16]],derivIndices=[2,3]))
rk.addCustomDerivation("filterMinusLargeK200", sc.rangeFilterDerivation("K200",controlIndices=[1],controlRanges=[[-1e16,-0.01]],derivIndices=[2,3]))

rk.addCustomDerivation("filterSmallValsK002", sc.rangeFilterDerivation("K002Small",controlIndices=[1],controlRanges=[[-0.01,0.01]],derivIndices=[2,3]))
rk.addCustomDerivation("filterPlusLargeK002", sc.rangeFilterDerivation("K002",controlIndices=[1],controlRanges=[[0.01,1e16]],derivIndices=[2,3]))
rk.addCustomDerivation("filterMinusLargeK002", sc.rangeFilterDerivation("K002",controlIndices=[1],controlRanges=[[-1e16,-0.01]],derivIndices=[2,3]))

Since the ranges of the filtered functions do not overlap, we can simply sum them up and store them as a single variable so that $K_{200}$ and $K_{002}$ exist over the range $-1e16 < X < 1e16$. This can be done using the `additiveDerivation()` function found in `simple_containers`.

The `additiveDerivation()` requires a list of the derivations which are to be summed and stored as a single variable. The result can be raised to an arbitrary power. Similarly to the `rangeFilterDerivation()`, the `additiveDerivation()` is passed a list of variables that are used within the derivation. The indecies of the variables required for each derivation must be provided.

This is also explained in the docstring of the `additiveDerivation()`:

In [38]:
help(sc.additiveDerivation)

Help on function additiveDerivation in module RMK_support.simple_containers:

additiveDerivation(derivTags: List[str], resultPower: float, derivIndices: List[List[int]], linCoeffs: List[float] = []) -> dict
    Returns property dictionary for additive composite derivation which sums up the results of each derivation in derivTags and
    raises the corresponding result to resultPower.
    
    Args:
        derivTags (List[str]): List of derivations whose output should be added
        resultPower (float): Power to raise the result of the addition
        derivIndices (List[List[int]]]): List of index lists corresponding to each derivation in derivTags.
        linCoeffs (List[List[int]]]): List linear coefficients corresponding to each derivation in derivTags. Defaults to [] resulting in a list of ones.
    
    Returns:
        dict: Dictionary representing derivation properties
    
    Usage:
        Given a passed set of variables to the additive derivation object, each individual 

We can now add the variables for $K_{200}$ and $K_{002}$, in all of the different regimes adressed above.

In [39]:
rk.addVarAndDual("K200",
                 isDerived=True,
                 derivationRule=sc.derivationRule("filteredK200",["X","TePar","TePerp"]),
                 derivOptions=sc.additiveDerivation(["filterSmallValsK200","filterPlusLargeK200","filterMinusLargeK200"],resultPower=1., derivIndices=[[1,2,3]]*3))

rk.addVarAndDual("K002",
                 isDerived=True,
                 derivationRule=sc.derivationRule("filteredK002",["X","TePar","TePerp"]),
                 derivOptions=sc.additiveDerivation(["filterSmallValsK002","filterPlusLargeK002","filterMinusLargeK002"],resultPower=1., derivIndices=[[1,2,3]]*3))

### Implicit temperature derivation
Fluid models in ReMKiT1D are mostly written in conservative form. As such, the temperature is considered a derived variable. However, we can make use of the `implicitTemperatures()` function found in `common_models` to derive the temperature from the implicit fluid variables such that temperature is also an implicit variable.

In [40]:
help(cm.implicitTemperatures)

Help on function implicitTemperatures in module RMK_support.common_models:

implicitTemperatures(modelTag: str, speciesFluxes: List[str], speciesEnergies: List[str], speciesDensities: List[str], speciesTemperatures: List[str], species: List[RMK_support.simple_containers.Species], speciesDensitiesDual: Optional[List[str]] = None, evolvedXU2Cells: Optional[List[int]] = None, ignoreKineticContribution=False, degreesOfFreedom: int = 3) -> RMK_support.simple_containers.CustomModel
    Generate implicit temperature derivation terms for each species
    
    Args:
        speciesFluxes (List[str]): Names of evolved species fluxes
        speciesEnergies (List[str]): Names of species energies
        speciesDensities (List[str]): Names of species densities
        speciesTemperatures (List[str]): Names of species temperature
        species (list[sc.Species]): Species objects for each species
        speciesDensitiesDual (Union[List[str],None], optional): Names of species densities on dual gri

In [41]:
# Initializing model
implicitTempModelPar = cm.implicitTemperatures(modelTag="implicitTempElPar",
                                            speciesFluxes=["Ge_dual"],
                                            speciesDensities=["ne"],
                                            speciesEnergies=["WePar"],
                                            speciesTemperatures=["TePar"],
                                            species=[electronSpecies],
                                            speciesDensitiesDual=["ne_dual"],
                                            degreesOfFreedom=1)

rk.addModel(implicitTempModelPar)

# Initializing model
implicitTempModelPerp = cm.implicitTemperatures(modelTag="implicitTempElPerp",
                                            speciesFluxes=["Ge_dual"],
                                            speciesDensities=["ne"],
                                            speciesEnergies=["WePerp"],
                                            speciesTemperatures=["TePerp"],
                                            species=[electronSpecies],
                                            speciesDensitiesDual=["ne_dual"],
                                            degreesOfFreedom=2,
                                            ignoreKineticContribution=True)

rk.addModel(implicitTempModelPerp)

Checking terms in model implicitTempElPar:
   Checking term identityTermTePar
   Checking term wTermTePar
   Checking term u2TermTePar
Checking terms in model implicitTempElPerp:
   Checking term identityTermTePerp
   Checking term wTermTePerp


### Temperature Isotropisation
We have a small number of evolved variables, with many custom terms, all with diagonal stencils. This is a prime candidate for the use of the `addNodeMatrixTermModel()` function to nearly create the models in single lines.

In [42]:
perpIsoEl = cm.addNodeMatrixTermModel(rk,
                                      modelTag="perpIsoEl", 
                                      evolvedVar="WePerp",
                                      termDefs=[(2*Node("collFreq")*Node("K002")/Node("TePerp"),"ne"),
                                                (-2*Node("collFreq")*Node("K200")/Node("TePerp"),"ne")]) # YOUR CODE HERE?

parIsoEl = cm.addNodeMatrixTermModel(rk,
                                      modelTag="parIsoEl", 
                                      evolvedVar="WePar",
                                      termDefs=[(4*Node("collFreq")*Node("TePerp")*Node("K200")/(Node("TePar")**2),"ne"),
                                                (-4*Node("collFreq")*Node("TePerp")*Node("K002")/(Node("TePar")**2),"ne")]) # YOUR CODE HERE?

Checking terms in model perpIsoEl:
   Checking term nodeTerm_0
   Checking term nodeTerm_1
Checking terms in model parIsoEl:
   Checking term nodeTerm_0
   Checking term nodeTerm_1


### Time integration

In [43]:
rk.addIntegrator("BE",sc.picardBDEIntegrator(nonlinTol=1e-12,absTol=10.0,convergenceVars=['TePerp','TePar']) ) # we want all evolved variables to converge

rk.setIntegratorGlobalData(initialTimestep=0.1)

rk.addTermDiagnosisForVars(["WePar","WePerp"])

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

rk.setFixedNumTimesteps(2500)
rk.setFixedStepOutput(50)

### Write config file

In [44]:
rk.writeConfigFile()

### Data analysis

In [45]:
numFiles = 50
loadFilenames = [hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames,
                             isXinMeters=rk.grid.isLengthInMeters)
loadedData

KeyError: "Unable to synchronously open object (object 'parIsoElnodeTerm_0' doesn't exist)"

In [ ]:
hv.extension('matplotlib')
%matplotlib inline
plt.rcParams['figure.dpi'] = 150
hv.output(size=100, dpi=150)

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [ ]:
pn.extension(comms="vscode")  # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData, rk.grid)

dashboard.fluid2Comparison().show()


Launching server at http://localhost:37657
